<a href="https://colab.research.google.com/github/KiaroRB/data-analysis-projects/blob/main/bronze_data_ingestion_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
      .appName("bronze_ingestion")\
      .getOrCreate()

In [5]:
!pip install -q gdown

In [6]:
import gdown

file_id = "1_0SQv1AFNNspJDRv1FgbWeVUBUqC5_Qb"
gdown.download(id=file_id, output="online_retail.csv", quiet=False)

#https://drive.google.com/file/d/1_0SQv1AFNNspJDRv1FgbWeVUBUqC5_Qb/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1_0SQv1AFNNspJDRv1FgbWeVUBUqC5_Qb
To: /content/online_retail.csv
100%|██████████| 45.6M/45.6M [00:00<00:00, 151MB/s]


'online_retail.csv'

In [7]:
df = spark.read.csv(
    "online_retail.csv",
    header=True,
    inferSchema=True,
    sep = ";"
)

In [17]:
df.write.mode("overwrite").parquet("bronze_online_retail_raw")

In [ ]:
print("Total de registros:", df.count())
print("\nTotal de columnas:", len(df.columns))
print("\n REGISTRO DEL DATASET")
df.limit(5).show(truncate=False)
print("\n Verificación de datos")
df.printSchema()

Total de registros: 541909

Total de columnas: 8

 REGISTRO DEL DATASET
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |1/12/2010 8:26|2,55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |1/12/2010 8:26|3,39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |1/12/2010 8:26|2,75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |1/12/2010 8:26|3,39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |1/12/2010 8:26|3,39     |17850     |

In [34]:
#ENTREGABLE 2 — Plata: Limpieza + columnas derivadas
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [36]:
total_bronze = df.count()
print("Registros antes de limpiar:", total_bronze)

Registros antes de limpiar: 541909


In [37]:
#Regla 1: Quantity > 0
from pyspark.sql.functions import col
df_quantity_ok = df.filter(col("Quantity") > 0)

df_quantity_ok.select("Quantity").show(5)

print("Registros luego de Quantity > 0:", df_quantity_ok.count())
print(
    "Eliminados por Quantity <= 0:",
    total_bronze - df_quantity_ok.count()
)

+--------+
|Quantity|
+--------+
|       6|
|       6|
|       8|
|       6|
|       6|
+--------+
only showing top 5 rows
Registros luego de Quantity > 0: 531285
Eliminados por Quantity <= 0: 10624


In [44]:
#Correción del tipo de dato UnitPrice (string a double)
from pyspark.sql.functions import regexp_replace

df_price_cast = df_quantity_ok.withColumn(
    "UnitPrice",
    regexp_replace(col("UnitPrice"), ",", ".").cast("double")
)

df_price_cast.printSchema()

#Regla 1: UnitPrice > 0
df_price_ok = df_price_cast.filter(col("UnitPrice") > 0)

print("Registros antes (Quantity OK):", df_quantity_ok.count())
print("Registros después (UnitPrice > 0):", df_price_ok.count())
print(
    "Eliminados por UnitPrice <= 0:",
    df_quantity_ok.count() - df_price_ok.count()
)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

Registros antes (Quantity OK): 531285
Registros después (UnitPrice > 0): 530104
Eliminados por UnitPrice <= 0: 1181
